# Type charts

By **Franklin Oliveira**

-----
This notebook contains all code necessary to make the "type" charts from `repteis` database. Here you'll find some basic data treatment and charts' code. 

Database: <font color='blue'>'Compilacao Livros Repteis - 2 a 10 - 2020_04_28.xls'</font>.

In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# quick visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Waffle Charts
# from pywaffle import Waffle 
# docs: https://pywaffle.readthedocs.io/en/latest/examples/block_shape_distance_location_and_direction.html

# visualization
import altair as alt

# enabling notebook renderer
# alt.renderers.enable('notebook')
alt.renderers.enable('default')

# disabling rows limit
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

In [2]:
excel = pd.ExcelFile('Compilacao Livros Repteis - 2 a 10 - 2020_04_28.xls')
sheet_name = excel.sheet_names

print('The excel file contains the following sheets:', sheet_name)
print('\nDatabase is in sheet:', sheet_name[0])

The excel file contains the following sheets: ['Repteis-2020-02-11-csv']

Database is in sheet: Repteis-2020-02-11-csv


In [3]:
db = excel.parse(sheet_name[0], sep=';', encoding='utf-8-sig')

#db = pd.read_excel('db.csv', sep=',', encoding='utf-8-sig', low_memory=False)

print(f'The database has {db.shape[0]} rows and {db.shape[1]} columns.')

The database has 23119 rows and 114 columns.


In [4]:
# copying database to another variable to make a few changes keeping the original intact
repteis = db.copy()

<br>

<font color='red' size='5'>**p.s.:** We were told by the Curator that some names in this database may remain confidential. So, I'm gonna skip this step for now until we know how to treat this data appropriately </font>

### Name columns

**Determiner:** `DeterminatorFirstName1` and `DeterminatorLastName1`

**Collector:** `CollectorFirstName1` and `CollectorLastName1`

#### treating determiner's name 

In this step, we're going to put together 'determinator's' first and last name. 

`DeterminatorFirstName1` + `DeterminatorLastName1`

Just concatenating those two columns is not satisfactory because there's some dirt in these data columns. So, let's begin by treating them...

In [6]:
def treat_names(name, pos='first'):
    '''
    Treat names keeping NaN as such.
    
    Arguments: 
        - name: name to be treated. 
        - pos (str): name position. One of ['first', 'last']
    '''
    if type(name) == str and pos == 'first':     # first name
        
        if len(name.split(' ')) > 1:             # checks if it's a composite name (+ 1 name)
            return str(name).strip().split(' ')[0].capitalize()         # keeps only 1st name
        else:
            return str(name).strip().capitalize()
    
    elif type(name) == str and pos == 'last':    # last name
        
        if len(name.split(' ')) > 1:             # treats composite name (+ 1 surname)
            return str(name).strip().split(' ')[-1].capitalize()
        else:
            return str(name).strip().capitalize()   
    else:
        return name

applying function to the two column names

In [7]:
repteis['DeterminatorFirstName1'] = repteis['DeterminatorFirstName1'].apply(treat_names)
repteis['CollectorFirstName1'] = repteis['CollectorFirstName1'].apply(treat_names)
# crustaceas['Determiner First Name1'].value_counts()

In [8]:
repteis['DeterminatorLastName1'] = repteis['DeterminatorLastName1'].apply(lambda x:
                                                                treat_names(x, pos='last'))

repteis['CollectorLastName1'] = repteis['CollectorLastName1'].apply(lambda x:
                                                                treat_names(x, pos='last'))

# crustaceas['Determiner Last Name1'].value_counts()

In [9]:
# creating column with First and Last name for identification
repteis['DeterminatorFirst_and_LastName'] = repteis['DeterminatorFirstName1'] + ' ' + repteis['DeterminatorLastName1']

repteis['CollectorFirst_and_LastName'] = repteis['CollectorFirstName1'] + ' ' + repteis['CollectorLastName1']

<br>

## Adjusting columns names

### removing '\n'

In [10]:
repteis.columns = [str(col).replace(r'\n','') for col in repteis.columns]

<br>

## preparing data for charts...

### Column: `Type Status 1`

In [11]:
repteis['Type Status 1'].value_counts().head()

Series([], Name: Type Status 1, dtype: int64)

#### Let's begin cleaning this data and lowering all cases.

In [12]:
# repteis['Type Status 1'] = repteis['Type Status 1'].str.strip().str.lower().str.capitalize()

In [13]:
# repteis['Type Status 1'].value_counts().head()

### preparing taxonomy columns

`Kingdom` - `Phylum` - `Class` - `Ordem` - `Familia` - `Genero_ent`- `Genero_atual` - `Especie_ent` - `Especie_atual`

**Colunas "faltando" (se comparado à base crustaceas):**
- `Subphylum1`
- `Subclass1`
- `Infraclass1`
- `Superorder1`
- `Suborder1` 
- `Infraorder1` 
- `Superfamily1`
- `Subfamily1` 
- `Tribe1`

In [14]:
taxon_columns = ['Kingdom', 'Phylum', 'Class', 'Ordem', 'Familia', 'Genero_ent',
                 'Genero_atual', 'Especie_ent', 'Especie_atual', 'Subespecie_ent',
                 'Subespecie_atual']  # selecting taxonomy columns

# defining function
def treat_str(x):
    return str(x).lower().capitalize().strip()

# applying treatment
for col in taxon_columns:
    print(f'Adjusting column {col}')
    repteis[col] = repteis[col].apply(treat_str)

Adjusting column Kingdom
Adjusting column Phylum
Adjusting column Class
Adjusting column Ordem
Adjusting column Familia
Adjusting column Genero_ent
Adjusting column Genero_atual
Adjusting column Especie_ent
Adjusting column Especie_atual
Adjusting column Subespecie_ent
Adjusting column Subespecie_atual


### adding `Genero` and `Especie` together (they completely identify each animal's species)

In [15]:
repteis['genero_e_especie_ent'] = repteis['Genero_ent'] + ' ' + repteis['Especie_ent']
repteis['genero_e_especie_atual'] = repteis['Genero_atual'] + ' ' + repteis['Especie_atual']

# tratamento (removendo espaços e padronizando letras)
repteis['genero_e_especie_ent'] = repteis['genero_e_especie_ent'].str.strip().str.lower().str.capitalize()
repteis['genero_e_especie_atual'] = repteis['genero_e_especie_atual'].str.strip().str.lower().str.capitalize()

<br>

### Collecting date (year) and sclicing main DB to a smaller dataset 

Columns: `Determined Date1` - `Class1` - `Kingdom` and more...

In [16]:
# slicing main database (repteis)
Table = repteis[['DataDeEntrada','DataDaDeterminacao','DataColetaInicial','Class','Kingdom', 
                    'Genero_ent', 'Genero_atual', 'Especie_ent', 'Especie_atual', 'Type Status 1',
                    'DeterminatorFirstName1', 'DeterminatorLastName1',
                    'DeterminatorFirst_and_LastName', 'CollectorFirstName1', 
                    'CollectorLastName1', 'CollectorFirst_and_LastName', 'genero_e_especie_ent',
                    'genero_e_especie_atual','MinAltitude',
                    'Ordem', 'Familia', 'Phylum']].copy()

# OBS: Determined Date1 has many missing values... CHECK THAT
d = []
counter=0
for row in Table['DataColetaInicial']:
    if not str(row).find('/')==-1:
        dates_values = str(row).split("/")
        year = int(dates_values[-1])
        month = int(dates_values[1])
#        if (month>1) and (month<12):
            #store the year and month in a datetime datatype for later sorting
#            dateRecord = datetime.datetime(year,month,1) 
    else:
        year = Table.loc[counter, 'DataColetaInicial']
    
    # mais um condicional para tratar anos vazios ' '
    if year == ' ':
        year = np.NAN
        
    d.append({'ano_coleta':year,
              'class':Table.loc[counter,'Class'],
              'kingdom':Table.loc[counter,'Kingdom'], 'genero_ent':Table.loc[counter,'Genero_ent'],
              'genero_atual':Table.loc[counter,'Genero_atual'],
              'especie_ent':Table.loc[counter,'Especie_ent'],
              'especie_atual':Table.loc[counter,'Especie_atual'],
              'genero_e_especie_ent': Table.loc[counter,'genero_e_especie_ent'],
              'genero_e_especie_atual': Table.loc[counter,'genero_e_especie_atual'],
              'type_status':Table.loc[counter,'Type Status 1'], 
              'determinator_first_name':Table.loc[counter,'DeterminatorFirstName1'],
              'determinator_last_name':Table.loc[counter,'DeterminatorLastName1'],
              'determinator_first_and_last_name': Table.loc[counter,'DeterminatorFirst_and_LastName'],
              'collector_first_name':Table.loc[counter,'CollectorFirstName1'],
              'collector_last_name':Table.loc[counter,'CollectorLastName1'],
              'collector_first_and_last_name': Table.loc[counter,'CollectorFirst_and_LastName'],
              'altitude':Table.loc[counter,'MinAltitude'],
              'ordem':Table.loc[counter,'Ordem'],
              'familia':Table.loc[counter,'Familia'],
              'phylum': Table.loc[counter,'Phylum']
              })
    counter = counter+1

    
NewTable = pd.DataFrame(d)


### collecting determined year (p.s.: being careful to keep NaNs as they show up)
NewTable['ano_determinacao'] = np.nan

d1 = []
counter=0
for row in Table['DataDaDeterminacao']:
    try:  # if Determined Date1 is empty, keep it so 
        if np.isnan(row):
            year= np.NAN
    
    except:
        if not str(row).find('/')==-1:
            dates_values = str(row).split("/")
            year = int(dates_values[-1])
            month = int(dates_values[1])
#            if (month>1) and (month<12):
                #store the year and month in a datetime datatype for later sorting
#                dateRecord = datetime.datetime(year,month,1)    
    
    NewTable.loc[counter, 'ano_determinacao'] = year
    counter = counter+1

    
### collecting start year (p.s.: being careful to keep NaNs as they show up)  
NewTable['ano_entrada'] = np.nan
d1 = []
counter=0
for row in Table['DataDeEntrada']:
    try:  # if Start Date is empty, keep it so 
        if np.isnan(row):
            year= np.NAN
    
    except:
        if not str(row).find('/')==-1:
            dates_values = str(row).split("/")
            year = int(dates_values[-1])
            month = int(dates_values[1])
#            if (month>1) and (month<12):
                #store the year and month in a datetime datatype for later sorting
#                dateRecord = datetime.datetime(year,month,1)    

    NewTable.loc[counter, 'ano_entrada'] = year
    counter = counter+1

# NewTable['determined_year'] = pd.Series(year, index=NewTable.index)
NewTable.head(2)

,ano_coleta,class,kingdom,genero_ent,genero_atual,especie_ent,especie_atual,genero_e_especie_ent,genero_e_especie_atual,type_status,...,determinator_first_and_last_name,collector_first_name,collector_last_name,collector_first_and_last_name,altitude,ordem,familia,phylum,ano_determinacao,ano_entrada
0,NaN,Reptilia,Animalia,Nan,Phrynops,Nan,Geoffroanus,Nan nan,Phrynops geoffroanus,NaN,...,L. Cunha,NaN,NaN,NaN,NaN,Testudines,Chelidae,Chordata,2018.0,NaN
1,NaN,Reptilia,Animalia,Heloderma,Heloderma,Suspectum,Suspectum,Heloderma suspectum,Heloderma suspectum,NaN,...,NaN,NaN,NaN,NaN,NaN,Squamata,Helodermatidae,Chordata,NaN,NaN


In [17]:
# checks if NaNs are in the same position 
result = (NewTable['ano_entrada'].isna() == repteis['DataDeEntrada'].isna()).sum() == NewTable.shape[0]

if result:
    print('ano_entrada info is valid.')
else:
    print("There's something wrong with NewTable. Check how you're collecting Start Year info.")

ano_entrada info is valid.


In [18]:
# year in which the holotipo was "firstly" found 
# NewTable['holotipo_year'] = NewTable['species_author'].str.extract('(\d+)')

<br>

### creating `years` columns in crustaceas

In [19]:
def catch_year(row):
    if not str(row).find('/')==-1:
        dates_values = str(row).split("/")
        year = int(dates_values[0])
        month = int(dates_values[1])
        return year
    else:
        return np.NaN

In [20]:
repteis['ano_determinacao'] = repteis['DataDaDeterminacao'].apply(catch_year)
repteis['ano_coleta'] = repteis['DataColetaInicial'].apply(catch_year)
repteis['ano_entrada'] = repteis['DataDeEntrada'].apply(catch_year)

<br>


## Graphs

---

### Counts  per researcher *per year*  (Determiner Name and Determined Year)

x: year, cor: types, size: counts

#### adjusting columns `ano_determinacao`, `ano_coleta` and `ano_entrada` to ```*int*``` format 

In [21]:
def str_with_nan2int(string):
    if not np.isnan(string):
        return int(string)
    else:
        return np.NAN

In [22]:
NewTable['ano_determinacao'] = NewTable['ano_determinacao'].apply(str_with_nan2int) #has NaN
NewTable['ano_coleta'] = NewTable['ano_coleta'].apply(str_with_nan2int) #has NaN
NewTable['ano_entrada'] = NewTable['ano_entrada'].apply(str_with_nan2int) #has NaN

#### determinator_full_name = determinator_first_name + determinator_last_name

In [23]:
NewTable['determinator_full_name'] = NewTable['determinator_first_name'] + ' ' + NewTable['determinator_last_name']

NewTable['collector_full_name'] = NewTable['collector_first_name'] + ' ' + NewTable['collector_last_name']

In [25]:
NewTable['collector_full_name'].value_counts()

A. Silveira      1003
U. Caramaschi     759
D. Vrcibradic     750
A. Carvalho       570
C. Rocha          515
                 ... 
A. Demoner          1
F. Forgo            1
N. Jr               1
M. Amélia           1
P. Langelo          1
Name: collector_full_name, Length: 1696, dtype: int64

In [26]:
NewTable.columns

Index(['ano_coleta', 'class', 'kingdom', 'genero_ent', 'genero_atual',
       'especie_ent', 'especie_atual', 'genero_e_especie_ent',
       'genero_e_especie_atual', 'type_status', 'determinator_first_name',
       'determinator_last_name', 'determinator_first_and_last_name',
       'collector_first_name', 'collector_last_name',
       'collector_first_and_last_name', 'altitude', 'ordem', 'familia',
       'phylum', 'ano_determinacao', 'ano_entrada', 'determinator_full_name',
       'collector_full_name'],
      dtype='object')

<br>

### Creating chart: counts per determiner per year

In [60]:
teste = NewTable.groupby(['determinator_full_name','ano_determinacao']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

In [61]:
g1 = alt.Chart(teste, width=800, height=2200).mark_circle().encode(
    x= alt.X('ano_determinacao', type='ordinal', title='Determined Year'),
    y= alt.Y('determinator_full_name', type='nominal', title='Determiner Name', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts', scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    tooltip= alt.Tooltip(['determinator_full_name', 'ano_determinacao', 'counts'])
)

g1.save('./graphs/determiner/counts_per_determiner.html')

g1

alt.Chart(...)

<font color='red' size='5'>mesmo gráfico, ordenando também pela ordem</font>

**OBS:** Note que

- as contagens mudam (porque estamos ordenando por um fator a mais)
- há pontos sobrepostos (semelhante ao que tinhamos para a base crustaceas - para um mesmo ano, um mesmo pesquisador descobriu animais de ordens/familias diferentes)

In [62]:
teste1 = NewTable.groupby(['determinator_full_name','ano_determinacao', 'ordem']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

In [63]:
g2 = alt.Chart(teste1, width=800, height=2200).mark_circle().encode(
    x= alt.X('ano_determinacao', type='ordinal', title='Determined Year'),
    y= alt.Y('determinator_full_name', type='nominal', title='Determiner Name', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('ordem', type="nominal", title="Order"),
    tooltip= alt.Tooltip(['determinator_full_name', 'ano_determinacao', 'counts', 'ordem'])
)

# saving chart
# g2.save('./graphs/determiner/counts_per_determiner-w-order.html')

g2

alt.Chart(...)

<br>

<font color='red'>**mesmo gráfico, agora ordenando pelo primeiro ano que o pesquisador aparece na base**</font>

In [64]:
# ordenando
teste1.sort_values(['ano_determinacao', 'determinator_full_name'], inplace=True)

# salvando ordem das entradas
sorting = list(teste1['determinator_full_name'].unique())

In [67]:
g2 = alt.Chart(teste1, width=800, height=2200).mark_circle().encode(
    x= alt.X('ano_determinacao', type='ordinal', title='Determined Year'),
    y= alt.Y('determinator_full_name', type='nominal', title='Determiner Name', 
            sort=sorting),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('ordem', type="nominal", title="Order"),
    tooltip= alt.Tooltip(['determinator_full_name', 'ano_determinacao', 'counts', 'ordem'])
)

# saving chart
# g2.save('./graphs/determiner/counts_per_determiner-c-ordem_reordenado.html')

g2

alt.Chart(...)

<br>

### Creating chart: counts per collector per year

<font color='red' size='5'> Collectors name is sensitive data. Do not publish it without curator's permission </font>

In [66]:
teste = NewTable.groupby(['collector_full_name','ano_coleta']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

In [39]:
g1 = alt.Chart(teste, width=800, height=2200).mark_circle().encode(
    x= alt.X('ano_coleta', type='ordinal', title='Collected Year'),
    y= alt.Y('collector_full_name', type='nominal', title='Collector Name', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts', scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    tooltip= alt.Tooltip(['collector_full_name', 'ano_coleta', 'counts'])
)

g1.save('./graphs/collector/counts_per_collector.html')

g1

alt.Chart(...)

<font color='red' size='5'>mesmo gráfico, ordenando também pela ordem</font>

**OBS:** Note que

- as contagens mudam (porque estamos ordenando por um fator a mais)
- há pontos sobrepostos (semelhante ao que tinhamos para a base crustaceas - para um mesmo ano, um mesmo pesquisador descobriu animais de ordens/familias diferentes)

In [40]:
teste1 = NewTable.groupby(['collector_full_name','ano_coleta', 'ordem']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

In [41]:
g2 = alt.Chart(teste1, width=800, height=2200).mark_circle().encode(
    x= alt.X('ano_coleta', type='ordinal', title='Collected Year'),
    y= alt.Y('collector_full_name', type='nominal', title='Collector Name', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('ordem', type="nominal", title="Order"),
    tooltip= alt.Tooltip(['collector_full_name', 'ano_coleta', 'counts', 'ordem'])
)

# saving chart
# g2.save('./graphs/collector/counts_per_collector-w-order.html')

g2

alt.Chart(...)

<br>

<font color='red'>**mesmo gráfico, agora ordenando pelo primeiro ano que o pesquisador aparece na base**</font>

In [56]:
# ordenando
teste1.sort_values(['ano_coleta', 'collector_full_name'], inplace=True)

# salvando ordem das entradas
sorting = list(teste1['collector_full_name'].unique())

In [59]:
g2 = alt.Chart(teste1, width=800, height=2200).mark_circle().encode(
    x= alt.X('ano_coleta', type='ordinal', title='Collected Year'),
    y= alt.Y('collector_full_name', type='nominal', title='Collector Name', 
            sort=alt.EncodingSortField('ano_coleta', op="min", order='ascending')),
    size= alt.Size('counts', type="quantitative",scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    color= alt.Color('ordem', type="nominal", title="Order"),
    tooltip= alt.Tooltip(['collector_full_name', 'ano_coleta', 'counts', 'ordem'])
)

# saving chart
# g2.save('./graphs/collector/counts_per_researcher-c-ordem_reordenado.html')

g2

alt.Chart(...)

<br>

**The end!**

-----